In [1]:
import numpy as np
import scipy as sp
import scipy.optimize
import teneva
from time import perf_counter as tpc
from teneva import accuracy_on_data, accuracy, erank
from numba import jit
from opt_einsum import contract

In [2]:
def rand_tensor_idx(I, r):
    d = I.shape[1]
    ns = np.max(I, axis=0) + 1
    return rand_tensor(d, ns, r)


def rand_tensor(d, ns, r):
    ranks = [1] + [r]*(d - 1) + [1]
    
    res = [np.random.rand(*shape) for shape in zip(ranks, ns, ranks[1:])]
        
    return res
    
    
@jit
def _poly_arr(x, n, a=-1, b=1):

    x = -1. + 2.*(x - a)/(b-a)
        
    res = np.ones((len(x), n))
    if n == 1:
        return res
    res[:, 1] = x
    for i in range(2, n):
        res[:, i] = 2*x*res[:, i-1] - res[:, i-2]
        
    return res


def poly_arr(x, n, a=-1, b=1):
    x = np.asarray(x)
    reduce_dim = x.ndim == 0
    if reduce_dim:
        x = x[None]
        
    res = _poly_arr(x, n, a=-1, b=1)
    if reduce_dim:
        res = res[0]
        
    return res

In [3]:
np.random.seed(42)
h = 30
d = 5
N = 100000
rank = 3

pow_p = 8


train = teneva.sample_lhs([h]*d, N)
test  = teneva.sample_lhs([h]*d, N)

train_x = train.astype(float)/(0.5*h) - 0.5
test_x = test.astype(float)/(0.5*h) - 0.5


Y0 = rand_tensor_idx(train, rank)
teneva.show(Y0)

Y0_spec = rand_tensor(d, [pow_p]*d, rank)
teneva.show(Y0_spec)


Y_train = scipy.optimize.rosen(train_x.T)
Y_test = scipy.optimize.rosen(test_x.T)

TT-tensor     5D : |30| |30| |30| |30| |30|
<rank>  =    3.0 :    \3/  \3/  \3/  \3/
TT-tensor     5D : |8| |8| |8| |8| |8|
<rank>  =    3.0 :   \3/ \3/ \3/ \3/


# Demo spectral

In [4]:
Y_als_sp = teneva.als_spectral(train_x,  Y_train, Y0_spec, lambda x: poly_arr(x, Y0_spec[0].shape[1], a=-1, b=1), log=True, nswp=100)

# pre | time:      0.717 | rank:   3.0 | 
#   1 | time:      3.940 | rank:   3.0 | eps: 1.1e+08 | 
#   2 | time:      5.871 | rank:   3.0 | eps: 1.0e+00 | 
#   3 | time:      7.761 | rank:   3.0 | eps: 1.0e+00 | 
#   4 | time:      9.692 | rank:   3.0 | eps: 6.4e-02 | 
#   5 | time:     11.746 | rank:   3.0 | eps: 2.1e-03 | 
#   6 | time:     13.704 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


In [4]:
Y_als_sp = teneva.als_spectral(train_x,  Y_train, Y0_spec, lambda x: poly_arr(x, Y0_spec[0].shape[1], a=-1, b=1), log=True, nswp=100)

# pre | time:      0.674 | rank:   3.0 | 
#   1 | time:      3.936 | rank:   3.0 | eps: 1.1e+08 | 
#   2 | time:      5.842 | rank:   3.0 | eps: 1.0e+00 | 
#   3 | time:      7.799 | rank:   3.0 | eps: 1.0e+00 | 
#   4 | time:      9.756 | rank:   3.0 | eps: 6.4e-02 | 
#   5 | time:     11.699 | rank:   3.0 | eps: 2.1e-03 | 
#   6 | time:     13.650 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


# Demo adaptive

In [4]:
Y_als = teneva.als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10, e_adap=1e-3, r=5)

# pre | time:      0.061 | rank:   3.0 | 
#   1 | time:      5.571 | rank:   5.0 | eps: 1.2e+02 | 
#   2 | time:      9.571 | rank:   5.0 | eps: 2.5e-01 | 
#   3 | time:     13.693 | rank:   5.0 | eps: 8.3e-02 | 
#   4 | time:     18.155 | rank:   5.0 | eps: 3.9e-03 | 
#   5 | time:     23.083 | rank:   4.8 | eps: 1.5e-04 | 
#   6 | time:     27.432 | rank:   3.0 | eps: 1.0e-05 | 
#   7 | time:     31.896 | rank:   3.0 | eps: 7.4e-07 | 
#   8 | time:     36.408 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


In [5]:
Y_als = teneva.als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10)

# pre | time:      0.039 | rank:   3.0 | 
#   1 | time:      0.282 | rank:   3.0 | eps: 1.2e+02 | 
#   2 | time:      0.541 | rank:   3.0 | eps: 8.1e-02 | 
#   3 | time:      0.797 | rank:   3.0 | eps: 4.0e-03 | 
#   4 | time:      1.062 | rank:   3.0 | eps: 2.3e-04 | 
#   5 | time:      1.329 | rank:   3.0 | eps: 2.1e-05 | 
#   6 | time:      1.591 | rank:   3.0 | eps: 2.2e-06 | 
#   7 | time:      1.856 | rank:   3.0 | eps: 2.5e-07 | 
#   8 | time:      2.121 | rank:   3.0 | eps: 3.4e-08 | 
#   9 | time:      2.392 | rank:   3.0 | eps: 2.4e-08 | 
#  10 | time:      2.657 | rank:   3.0 | eps: 1.7e-08 | stop: nswp | 


In [8]:
Y_als = teneva.als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10, adaptive=True, maxr=5, eps_adap=1e-3)

# pre | time:      0.063 | rank:   3.0 | 
#   1 | time:      3.766 | rank:   5.0 | eps: 1.2e+02 | 
#   2 | time:      7.485 | rank:   5.0 | eps: 2.5e-01 | 
#   3 | time:     11.209 | rank:   5.0 | eps: 8.3e-02 | 
#   4 | time:     14.938 | rank:   5.0 | eps: 3.9e-03 | 
#   5 | time:     18.657 | rank:   4.8 | eps: 1.5e-04 | 
#   6 | time:     22.252 | rank:   3.0 | eps: 1.0e-05 | 
#   7 | time:     25.830 | rank:   3.0 | eps: 7.4e-07 | 
#   8 | time:     29.409 | rank:   3.0 | eps: 2.7e-08 | 
#   9 | time:     32.999 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


In [4]:
print('\n--- Base')
Y1 = teneva.als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10)
print('\n--- Rank-adaptive')
Y2 = teneva.als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10, e_adap=1e-3, r=5)
print('\n--- Result')
teneva.accuracy(Y1, Y2)


--- Base
# pre | time:      0.067 | rank:   3.0 | 
#   1 | time:      1.598 | rank:   3.0 | eps: 1.2e+02 | 
#   2 | time:      1.841 | rank:   3.0 | eps: 8.1e-02 | 
#   3 | time:      2.089 | rank:   3.0 | eps: 4.0e-03 | 
#   4 | time:      2.330 | rank:   3.0 | eps: 2.3e-04 | 
#   5 | time:      2.560 | rank:   3.0 | eps: 2.1e-05 | 
#   6 | time:      2.802 | rank:   3.0 | eps: 2.2e-06 | 
#   7 | time:      3.042 | rank:   3.0 | eps: 2.5e-07 | 
#   8 | time:      3.285 | rank:   3.0 | eps: 2.6e-08 | 
#   9 | time:      3.539 | rank:   3.0 | eps: 0.0e+00 | stop: e | 

--- Rank-adaptive
# pre | time:      0.048 | rank:   3.0 | 
#   1 | time:      4.330 | rank:   5.0 | eps: 1.2e+02 | 
#   2 | time:      9.107 | rank:   5.0 | eps: 2.5e-01 | 
#   3 | time:     13.445 | rank:   5.0 | eps: 8.3e-02 | 
#   4 | time:     18.155 | rank:   5.0 | eps: 3.9e-03 | 
#   5 | time:     22.694 | rank:   4.8 | eps: 1.5e-04 | 
#   6 | time:     27.191 | rank:   3.0 | eps: 1.0e-05 | 
#   7 | time:     32.9

0.0